In [32]:
import torch
import pickle
import random
import pandas as pd
import numpy as np

import main_regression as mr

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def regression_report(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred) 
    mse = mean_squared_error(y_true, y_pred) 
    rmse = np.sqrt(mse)
      
    print('The regression reports are as follows = ')
    print('MAE: ', round(mae,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(rmse,4))
    return mae, mse, rmse

In [33]:
# seed 고정
random_seed = 42

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [34]:
# Case 1. LSTM model (w/o data representation)
config1 = {
        'model': 'LSTM', # classification에 활용할 알고리즘 정의, {'LSTM', 'GRU', 'CNN_1D', 'LSTM_FCNs', 'FC'} 중 택 1
        'training': True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
        'best_model_path': './ckpt/lstm.pt',  # 학습 완료 모델 저장 경로
        'parameter': {
            'input_size': 24,  # 데이터의 변수 개수, int
            'num_layers': 2,  # recurrent layers의 수, int(default: 2, 범위: 1 이상)
            'hidden_size': 64,  # hidden state의 차원, int(default: 64, 범위: 1 이상)
            'dropout': 0.1,  # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
            'bidirectional': True,  # 모델의 양방향성 여부, bool(default: True)
            'num_epochs': 1000,  # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
            'batch_size': 16,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
            'lr': 0.0001,  # learning rate, float(default: 0.001, 범위: 0.1 이하)
            'device': 'cuda'  # 학습 환경, ["cuda", "cpu"] 중 선택
        }
}

# Case 2. GRU model (w/o data representation)
config2 = {
        'model': 'GRU', # classification에 활용할 알고리즘 정의, {'LSTM', 'GRU', 'CNN_1D', 'LSTM_FCNs', 'FC'} 중 택 1
        'training': True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
        'best_model_path': './ckpt/gru.pt',  # 학습 완료 모델 저장 경로
        'parameter': {
            'input_size': 24,  # 데이터의 변수 개수, int
            'num_layers': 2,  # recurrent layers의 수, int(default: 2, 범위: 1 이상)
            'hidden_size': 64,  # hidden state의 차원, int(default: 64, 범위: 1 이상)
            'dropout': 0.1,  # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
            'bidirectional': True,  # 모델의 양방향성 여부, bool(default: True)
            'num_epochs': 1000,  # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
            'batch_size': 16,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
            'lr': 0.0001,  # learning rate, float(default: 0.001, 범위: 0.1 이하)
            'device': 'cuda'  # 학습 환경, ["cuda", "cpu"] 중 선택
        }
}

# Case 3. CNN_1D model (w/o data representation)
config3 = {
        'model': 'CNN_1D', # classification에 활용할 알고리즘 정의, {'LSTM', 'GRU', 'CNN_1D', 'LSTM_FCNs', 'FC'} 중 택 1
        'training': True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
        'best_model_path': './ckpt/cnn_1d.pt',  # 학습 완료 모델 저장 경로
        'parameter': {
            'input_size': 24,  # 데이터의 변수 개수, int
            'seq_len': 144,  # 데이터의 시간 길이, int
            'output_channels': 64, # convolution layer의 output channel, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
            'kernel_size': 3, # convolutional layer의 filter 크기, int(default: 3, 범위: 3 이상, 홀수로 설정 권장)
            'stride': 1, # convolution layer의 stride 크기, int(default: 1, 범위: 1 이상)
            'padding': 0, # padding 크기, int(default: 0, 범위: 0 이상)
            'drop_out': 0.1, # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
            'num_epochs': 1000,  # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
            'batch_size': 16,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
            'lr': 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
            'device': 'cuda'  # 학습 환경, ["cuda", "cpu"] 중 선택
        }
}

# Case 4. LSTM_FCNs model (w/o data representation)
config4 = {
        'model': 'LSTM_FCNs', # classification에 활용할 알고리즘 정의, {'LSTM', 'GRU', 'CNN_1D', 'LSTM_FCNs', 'FC'} 중 택 1
        'training': True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
        'best_model_path': './ckpt/lstm_fcn.pt',  # 학습 완료 모델 저장 경로
        'parameter': {
            'input_size': 24,  # 데이터의 변수 개수, int
            'num_layers': 2,  # recurrent layers의 수, int(default: 1, 범위: 1 이상)
            'lstm_drop_out': 0.1, # LSTM dropout 확률, float(default: 0.4, 범위: 0 이상 1 이하)
            'fc_drop_out': 0.1, # FC dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
            'num_epochs': 1000, # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
            'batch_size': 16,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
            'lr': 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
            'device': 'cuda'  # 학습 환경, ["cuda", "cpu"] 중 선택
        }
}

# Case 5. fully-connected layers (w/ data representation)
config5 = {
        'model': 'FC', # classification에 활용할 알고리즘 정의, {'LSTM', 'GRU', 'CNN_1D', 'LSTM_FCNs', 'FC'} 중 택 1
        "training": True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
        "best_model_path": './ckpt/fc.pt',  # 학습 완료 모델 저장 경로
        'parameter': {
            'input_size': 64,  # 데이터의 변수 개수(representation 차원), int
            'drop_out': 0.1, # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
            'bias': True, # bias 사용 여부, bool(default: True)
            'num_epochs': 1000, # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
            'batch_size': 16,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
            'lr': 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
            'device': 'cuda'  # 학습 환경, ["cuda", "cpu"] 중 선택
        }
}

In [35]:
# raw time series data
train_x = pickle.load(open('./data/x_train.pkl', 'rb'))
train_y = pickle.load(open('./data/y_train.pkl', 'rb'))
test_x = pickle.load(open('./data/x_test.pkl', 'rb'))
test_y = pickle.load(open('./data/y_test.pkl', 'rb'))

print(train_x.shape)  #shape : (num_of_instance x input_dims x window_size) = (95, 24, 144)
print(train_y.shape) #shape : (num_of_instance) = (95,)
print(test_x.shape)  #shape : (num_of_instance x input_dims x window_size) = (42, 24, 144)
print(test_y.shape)  #shape : (num_of_instance) = (42,)

(95, 24, 144)
(95,)
(42, 24, 144)
(42,)


In [36]:
# scaling time series data
scaler = MinMaxScaler()

if len(train_x.shape) < 3:
    scaler = scaler.fit(train_x)
else:
    origin_shape = train_x.shape
    scaler = scaler.fit(np.transpose(train_x, (0, 2, 1)).reshape(-1, origin_shape[1]))

scaled_x_data = []
for x_data in [train_x, test_x]:
    if len(train_x.shape) < 3:
        x_data = scaler.transform(x_data)
    else:
        x_data = scaler.transform(np.transpose(x_data, (0, 2, 1)).reshape(-1, origin_shape[1]))
        x_data = np.transpose(x_data.reshape(-1, origin_shape[2], origin_shape[1]), (0, 2, 1))

    scaled_x_data.append(x_data)
train_x, test_x = scaled_x_data        

print(train_x.shape)  #shape : (num_of_instance x input_dims x window_size) = (95, 24, 144)
print(test_x.shape)  #shape : (num_of_instance x input_dims x window_size) = (42, 24, 144)

(95, 24, 144)
(42, 24, 144)


In [37]:
# normalization of Y data
y_scaler = MinMaxScaler()
y_scaler = y_scaler.fit(train_y.reshape(train_y.shape[0], -1))

train_y = y_scaler.transform(train_y.reshape(train_y.shape[0], -1)).flatten()
test_y = y_scaler.transform(test_y.reshape(test_y.shape[0], -1)).flatten()

print(train_y.shape) #shape : (num_of_instance) = (95,)
print(test_y.shape)  #shape : (num_of_instance) = (42,)

(95,)
(42,)


In [38]:
train_data = {'x': train_x, 'y': train_y}
test_data = {'x': test_x, 'y': test_y}

In [39]:
# Case 1. LSTM model (w/o data representation)
config = config1
data_reg = mr.Regression(config, train_data, test_data)
model = data_reg.build_model()  # 모델 구축

if config["training"]:
    best_model = data_reg.train_model(model)  # 모델 학습
    data_reg.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

pred, mse, mae = data_reg.pred_data(model, best_model_path=config["best_model_path"])  # 예측

Start training model

Epoch 1/1000
train Loss: 0.2106
val Loss: 0.1529

Epoch 10/1000
train Loss: 0.0725
val Loss: 0.0391

Epoch 20/1000
train Loss: 0.0422
val Loss: 0.0319

Epoch 30/1000
train Loss: 0.0410
val Loss: 0.0327

Epoch 40/1000
train Loss: 0.0398
val Loss: 0.0330

Epoch 50/1000
train Loss: 0.0379
val Loss: 0.0350

Epoch 60/1000
train Loss: 0.0345
val Loss: 0.0364

Epoch 70/1000
train Loss: 0.0283
val Loss: 0.0402

Epoch 80/1000
train Loss: 0.0246
val Loss: 0.0394

Epoch 90/1000
train Loss: 0.0223
val Loss: 0.0412

Epoch 100/1000
train Loss: 0.0198
val Loss: 0.0449

Epoch 110/1000
train Loss: 0.0187
val Loss: 0.0426

Epoch 120/1000
train Loss: 0.0158
val Loss: 0.0432

Epoch 130/1000
train Loss: 0.0147
val Loss: 0.0436

Epoch 140/1000
train Loss: 0.0133
val Loss: 0.0488

Epoch 150/1000
train Loss: 0.0134
val Loss: 0.0416

Epoch 160/1000
train Loss: 0.0116
val Loss: 0.0407

Epoch 170/1000
train Loss: 0.0111
val Loss: 0.0418

Epoch 180/1000
train Loss: 0.0105
val Loss: 0.0405

E

In [40]:
print(f'** Performance of test dataset ==> MSE = {mse}, MAE = {mae}')
print(f'\n** Dimension of result for test dataset = {pred.shape}')
print(f'\n** Result for test dataset = \n{pred}')

** Performance of test dataset ==> MSE = 0.017429009079933167, MAE = 0.10097896307706833

** Dimension of result for test dataset = (42,)

** Result for test dataset = 
[0.3047709  0.42683783 0.35981247 0.3466163  0.29597327 0.28176564
 0.72853875 0.3096343  0.34975728 0.31722355 0.51728046 0.33553642
 0.37140888 0.4732644  0.39515972 0.58336943 0.29944944 0.5012954
 0.2999904  0.40493697 0.25795576 0.4667928  0.30028078 0.4635774
 0.31464398 0.3430924  0.3346968  0.6965303  0.45940176 0.2944054
 0.79077196 0.26151785 0.44176152 0.26427045 0.40654683 0.30194065
 0.3687495  0.5474853  0.5593459  0.25312018 0.23443124 0.60113305]


In [41]:
test_y_inverse = y_scaler.inverse_transform(test_y.reshape(test_y.shape[0], -1)).flatten()
pred_y_inverse = y_scaler.inverse_transform(pred.reshape(pred.shape[0], -1)).flatten()
mae, mse, rmse = regression_report (test_y_inverse, pred_y_inverse)

print(f'\n** Result for test dataset (inverse) = \n{pred_y_inverse}')

The regression reports are as follows = 
MAE:  2.1145
MSE:  7.6423
RMSE:  2.7645

** Result for test dataset (inverse) = 
[11.721903  14.277985  12.874474  12.5981455 11.537681  11.240173
 20.595602  11.823743  12.663918  11.982661  16.171852  12.366134
 13.117303  15.250157  13.614644  17.555758  11.610471  15.837127
 11.6217985 13.819381  10.741594  15.114641  11.62788   15.04731
 11.928644  12.524355  12.348552  19.925344  14.959873  11.50485
 21.898766  10.816184  14.590487  10.873823  13.85309   11.662638
 13.061614  16.804342  17.052704  10.640336  10.24899   17.927727 ]


In [42]:
# Case 2. GRU (w/o data representation)
config = config2
data_reg = mr.Regression(config, train_data, test_data)
model = data_reg.build_model()  # 모델 구축

if config["training"]:
    best_model = data_reg.train_model(model)  # 모델 학습
    data_reg.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

pred, mse, mae = data_reg.pred_data(model, best_model_path=config["best_model_path"])  # 예측

Start training model

Epoch 1/1000
train Loss: 0.4116
val Loss: 0.2935

Epoch 10/1000
train Loss: 0.0459
val Loss: 0.0313

Epoch 20/1000
train Loss: 0.0434
val Loss: 0.0329

Epoch 30/1000
train Loss: 0.0423
val Loss: 0.0327

Epoch 40/1000
train Loss: 0.0411
val Loss: 0.0328

Epoch 50/1000
train Loss: 0.0399
val Loss: 0.0327

Epoch 60/1000
train Loss: 0.0387
val Loss: 0.0324

Epoch 70/1000
train Loss: 0.0373
val Loss: 0.0328

Epoch 80/1000
train Loss: 0.0360
val Loss: 0.0325

Epoch 90/1000
train Loss: 0.0347
val Loss: 0.0331

Epoch 100/1000
train Loss: 0.0334
val Loss: 0.0331

Epoch 110/1000
train Loss: 0.0318
val Loss: 0.0329

Epoch 120/1000
train Loss: 0.0306
val Loss: 0.0322

Epoch 130/1000
train Loss: 0.0290
val Loss: 0.0329

Epoch 140/1000
train Loss: 0.0277
val Loss: 0.0311

Epoch 150/1000
train Loss: 0.0269
val Loss: 0.0318

Epoch 160/1000
train Loss: 0.0250
val Loss: 0.0324

Epoch 170/1000
train Loss: 0.0240
val Loss: 0.0333

Epoch 180/1000
train Loss: 0.0239
val Loss: 0.0296

E

In [43]:
print(f'** Performance of test dataset ==> MSE = {mse}, MAE = {mae}')
print(f'\n** Dimension of result for test dataset = {pred.shape}')
print(f'\n** Result for test dataset = \n{pred}')

** Performance of test dataset ==> MSE = 0.018103018403053284, MAE = 0.10689087957143784

** Dimension of result for test dataset = (42,)

** Result for test dataset = 
[0.36939186 0.46465832 0.33999524 0.24856427 0.26177135 0.21089107
 0.7682883  0.35205546 0.25180808 0.3126573  0.62921196 0.2842126
 0.29155764 0.3062406  0.43172085 0.544011   0.48174143 0.26652867
 0.1900303  0.3990894  0.24132988 0.51944596 0.27525833 0.34875762
 0.365018   0.3091969  0.41964355 0.602382   0.2528928  0.23492199
 0.675107   0.21797833 0.40153563 0.26168284 0.5106204  0.17388318
 0.2152636  0.44397503 0.5998846  0.11629543 0.12592539 0.5139011 ]


In [44]:
test_y_inverse = y_scaler.inverse_transform(test_y.reshape(test_y.shape[0], -1)).flatten()
pred_y_inverse = y_scaler.inverse_transform(pred.reshape(pred.shape[0], -1)).flatten()
mae, mse, rmse = regression_report (test_y_inverse, pred_y_inverse)

print(f'\n** Result for test dataset (inverse) = \n{pred_y_inverse}')

The regression reports are as follows = 
MAE:  2.2383
MSE:  7.9379
RMSE:  2.8174

** Result for test dataset (inverse) = 
[13.075067  15.069946  12.4595    10.544936  10.821492   9.75606
 21.427958  12.712042  10.612862  11.887045  18.5157    11.291411
 11.445217  11.752678  14.380235  16.73159   15.427666  10.921111
  9.319235  13.696933  10.393448  16.2172    11.1039095 12.642984
 12.983477  11.814584  14.1273365 17.95388   10.635576  10.259267
 19.47674    9.904467  13.748156  10.819639  16.032393   8.981113
  9.84762   14.636838  17.901585   7.7752266  7.976878  16.10109  ]


In [45]:
# Case 3. CNN_1D (w/o data representation)
config = config3
data_reg = mr.Regression(config, train_data, test_data)
model = data_reg.build_model()  # 모델 구축

if config["training"]:
    best_model = data_reg.train_model(model)  # 모델 학습
    data_reg.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

pred, mse, mae = data_reg.pred_data(model, best_model_path=config["best_model_path"])  # 예측

Start training model

Epoch 1/1000
train Loss: 0.2695
val Loss: 0.1689

Epoch 10/1000
train Loss: 0.0459
val Loss: 0.0335

Epoch 20/1000
train Loss: 0.0421
val Loss: 0.0328

Epoch 30/1000
train Loss: 0.0389
val Loss: 0.0345

Epoch 40/1000
train Loss: 0.0375
val Loss: 0.0333

Epoch 50/1000
train Loss: 0.0355
val Loss: 0.0329

Epoch 60/1000
train Loss: 0.0341
val Loss: 0.0341

Epoch 70/1000
train Loss: 0.0311
val Loss: 0.0360

Epoch 80/1000
train Loss: 0.0260
val Loss: 0.0366

Epoch 90/1000
train Loss: 0.0288
val Loss: 0.0368

Epoch 100/1000
train Loss: 0.0239
val Loss: 0.0449

Epoch 110/1000
train Loss: 0.0209
val Loss: 0.0421

Epoch 120/1000
train Loss: 0.0194
val Loss: 0.0495

Epoch 130/1000
train Loss: 0.0194
val Loss: 0.0481

Epoch 140/1000
train Loss: 0.0165
val Loss: 0.0511

Epoch 150/1000
train Loss: 0.0168
val Loss: 0.0502

Epoch 160/1000
train Loss: 0.0152
val Loss: 0.0570

Epoch 170/1000
train Loss: 0.0138
val Loss: 0.0526

Epoch 180/1000
train Loss: 0.0149
val Loss: 0.0519

E

In [46]:
print(f'** Performance of test dataset ==> MSE = {mse}, MAE = {mae}')
print(f'\n** Dimension of result for test dataset = {pred.shape}')
print(f'\n** Result for test dataset = \n{pred}')

** Performance of test dataset ==> MSE = 0.02758614718914032, MAE = 0.13107439875602722

** Dimension of result for test dataset = (42,)

** Result for test dataset = 
[0.44077972 0.4256018  0.3758379  0.4256594  0.42308784 0.39099947
 0.3928827  0.46756265 0.4121578  0.46640396 0.40377593 0.43973404
 0.42629552 0.5247498  0.4278915  0.4313423  0.38731903 0.45402232
 0.39701858 0.45266867 0.3863931  0.41109562 0.42002586 0.39317295
 0.3822829  0.38881293 0.46851036 0.45563018 0.43082944 0.4291013
 0.46020874 0.34560657 0.4141638  0.388745   0.3378096  0.3645021
 0.42241335 0.55821687 0.36535075 0.3813812  0.3863528  0.43012348]


In [47]:
test_y_inverse = y_scaler.inverse_transform(test_y.reshape(test_y.shape[0], -1)).flatten()
pred_y_inverse = y_scaler.inverse_transform(pred.reshape(pred.shape[0], -1)).flatten()
mae, mse, rmse = regression_report (test_y_inverse, pred_y_inverse)

print(f'\n** Result for test dataset (inverse) = \n{pred_y_inverse}')

The regression reports are as follows = 
MAE:  2.7447
MSE:  12.0961
RMSE:  3.4779

** Result for test dataset (inverse) = 
[14.569927  14.252102  13.210046  14.253308  14.199459  13.527529
 13.566963  15.130762  13.970585  15.106499  13.795068  14.548032
 14.266628  16.328262  14.300048  14.372309  13.450461  14.847228
 13.653569  14.818882  13.431072  13.948342  14.135342  13.573042
 13.345005  13.481743  15.150607  14.880896  14.361568  14.325382
 14.976771  12.577002  14.01259   13.480322  12.4137335 12.972674
 14.185335  17.029062  12.990445  13.326123  13.430228  14.346786 ]


In [48]:
# Case 4. LSTM_FCNs (w/o data representation)
config = config4
data_reg = mr.Regression(config, train_data, test_data)
model = data_reg.build_model()  # 모델 구축

if config["training"]:
    best_model = data_reg.train_model(model)  # 모델 학습
    data_reg.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

pred, mse, mae = data_reg.pred_data(model, best_model_path=config["best_model_path"])  # 예측

Start training model

Epoch 1/1000
train Loss: 0.1856
val Loss: 0.1621

Epoch 10/1000
train Loss: 0.0465
val Loss: 0.0483

Epoch 20/1000
train Loss: 0.0132
val Loss: 0.0362

Epoch 30/1000
train Loss: 0.0076
val Loss: 0.0363

Epoch 40/1000
train Loss: 0.0063
val Loss: 0.0335

Epoch 50/1000
train Loss: 0.0062
val Loss: 0.0318

Epoch 60/1000
train Loss: 0.0022
val Loss: 0.0310

Epoch 70/1000
train Loss: 0.0048
val Loss: 0.0393

Epoch 80/1000
train Loss: 0.0021
val Loss: 0.0353

Epoch 90/1000
train Loss: 0.0075
val Loss: 0.0426

Epoch 100/1000
train Loss: 0.0015
val Loss: 0.0387

Epoch 110/1000
train Loss: 0.0019
val Loss: 0.0341

Epoch 120/1000
train Loss: 0.0018
val Loss: 0.0381

Epoch 130/1000
train Loss: 0.0031
val Loss: 0.0356

Epoch 140/1000
train Loss: 0.0045
val Loss: 0.0433

Epoch 150/1000
train Loss: 0.0079
val Loss: 0.0457

Epoch 160/1000
train Loss: 0.0031
val Loss: 0.0375

Epoch 170/1000
train Loss: 0.0026
val Loss: 0.0373

Epoch 180/1000
train Loss: 0.0067
val Loss: 0.0370

E

In [49]:
print(f'** Performance of test dataset ==> MSE = {mse}, MAE = {mae}')
print(f'\n** Dimension of result for test dataset = {pred.shape}')
print(f'\n** Result for test dataset = \n{pred}')

** Performance of test dataset ==> MSE = 0.021918687969446182, MAE = 0.11415926367044449

** Dimension of result for test dataset = (42,)

** Result for test dataset = 
[0.41571447 0.36990875 0.4027995  0.31158298 0.35978344 0.2612773
 0.5857863  0.37377116 0.33932647 0.49704972 0.49419686 0.32649043
 0.36809796 0.552688   0.4349853  0.52439344 0.4641516  0.4960039
 0.20722315 0.4218952  0.20041293 0.39132303 0.37325558 0.40476045
 0.35932904 0.39706486 0.32728073 0.47955844 0.25320253 0.40826923
 0.5054712  0.26766875 0.5665003  0.20846343 0.43009248 0.40670663
 0.32289103 0.66277146 0.5870684  0.18357942 0.48000178 0.51906145]


In [50]:
test_y_inverse = y_scaler.inverse_transform(test_y.reshape(test_y.shape[0], -1)).flatten()
pred_y_inverse = y_scaler.inverse_transform(pred.reshape(pred.shape[0], -1)).flatten()
mae, mse, rmse = regression_report (test_y_inverse, pred_y_inverse)

print(f'\n** Result for test dataset (inverse) = \n{pred_y_inverse}')

The regression reports are as follows = 
MAE:  2.3905
MSE:  9.611
RMSE:  3.1002

** Result for test dataset (inverse) = 
[14.045061  13.08589   13.774621  11.864549  12.873865  10.811147
 17.606365  13.166768  12.445497  15.748221  15.688482  12.17671
 13.047973  16.913286  14.448593  16.320799  15.059335  15.726322
  9.679253  14.174486   9.536647  13.534306  13.1559725 13.815684
 12.864351  13.654539  12.193259  15.381954  10.642061  13.889158
 15.924567  10.944984  17.202517   9.705224  14.346137  13.856438
 12.101338  19.218433  17.633213   9.184154  15.391237  16.209146 ]


--------------------------------------------------

In [51]:
# representation data
train_x = pd.read_csv('./data/ts2vec_repr_train.csv')
train_y = pickle.load(open('./data/y_train.pkl', 'rb'))

test_x = pd.read_csv('./data/ts2vec_repr_test.csv')
test_y = pickle.load(open('./data/y_test.pkl', 'rb'))

train_data = {'x': train_x, 'y': train_y}
test_data = {'x': test_x, 'y': test_y}

print(train_x.shape)  #shape : (num_of_instance x representation_dims) = (95, 64)
print(train_y.shape) #shape : (num_of_instance) = (95, )
print(test_x.shape)  #shape : (num_of_instance x representation_dims) = (42, 64)
print(test_y.shape)  #shape : (num_of_instance) = (42, )

(95, 64)
(95,)
(42, 64)
(42,)


In [52]:
# Case 5. fully-connected layers (w/ data representation)
config = config5
data_reg = mr.Regression(config, train_data, test_data)
model = data_reg.build_model()  # 모델 구축

if config["training"]:
    best_model = data_reg.train_model(model)  # 모델 학습
    data_reg.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

pred, mse, mae = data_reg.pred_data(model, best_model_path=config["best_model_path"])  # 예측

Start training model

Epoch 1/1000
train Loss: 209.6828
val Loss: 175.2797

Epoch 10/1000
train Loss: 192.5818
val Loss: 161.3480

Epoch 20/1000
train Loss: 173.0780
val Loss: 142.3571

Epoch 30/1000
train Loss: 147.4441
val Loss: 120.1095

Epoch 40/1000
train Loss: 118.7403
val Loss: 97.4779

Epoch 50/1000
train Loss: 95.7690
val Loss: 75.5590

Epoch 60/1000
train Loss: 73.7453
val Loss: 56.3923

Epoch 70/1000
train Loss: 56.7339
val Loss: 40.8764

Epoch 80/1000
train Loss: 44.2354
val Loss: 29.4787

Epoch 90/1000
train Loss: 34.6329
val Loss: 21.7455

Epoch 100/1000
train Loss: 31.0377
val Loss: 16.9953

Epoch 110/1000
train Loss: 28.1335
val Loss: 14.2482

Epoch 120/1000
train Loss: 25.8113
val Loss: 12.8126

Epoch 130/1000
train Loss: 23.5047
val Loss: 12.1300

Epoch 140/1000
train Loss: 22.8712
val Loss: 11.7650

Epoch 150/1000
train Loss: 24.5383
val Loss: 11.6133

Epoch 160/1000
train Loss: 20.0763
val Loss: 11.5351

Epoch 170/1000
train Loss: 21.8304
val Loss: 11.4859

Epoch 18

In [53]:
print(f'** Performance of test dataset ==> MSE = {mse}, MAE = {mae}')
print(f'\n** Dimension of result for test dataset = {pred.shape}')
print(f'\n** Result for test dataset = \n{pred}')

** Performance of test dataset ==> MSE = 14.826444625854492, MAE = 3.108508825302124

** Dimension of result for test dataset = (42,)

** Result for test dataset = 
[13.576086  14.051125  13.788996  11.888471  16.93226   12.359262
 11.974563  15.386085  13.054042  12.623264  12.654381  16.077137
 13.450314  15.690181  16.198381  13.821231  13.064769  13.32776
 12.786972  13.3713255 13.498327  12.48228   15.992512  15.445408
 15.823911  11.946144  12.971921  13.934993  12.8658285 13.864472
 13.1964655 12.532198  16.855518  16.06662   15.764783  12.323561
 12.073018  14.434633  11.96619   12.237625  13.90027   14.222074 ]


In [54]:
test_y_inverse = y_scaler.inverse_transform(test_y.reshape(test_y.shape[0], -1)).flatten()
pred_y_inverse = y_scaler.inverse_transform(pred.reshape(pred.shape[0], -1)).flatten()
mae, mse, rmse = regression_report (test_y_inverse, pred_y_inverse)

print(f'\n** Result for test dataset (inverse) = \n{pred_y_inverse}')

The regression reports are as follows = 
MAE:  65.0922
MSE:  6501.1523
RMSE:  80.6297

** Result for test dataset (inverse) = 
[289.62326 299.57056 294.08157 254.28458 359.90155 264.14297 256.08734
 327.52463 278.69165 269.67117 270.32275 341.99527 286.98956 333.8924
 344.53412 294.7566  278.91626 284.4233  273.0992  285.33557 287.995
 266.71893 340.2232  328.76685 336.6927  255.49226 276.97205 297.13876
 274.75046 295.66205 281.67398 267.76422 358.29456 341.77502 335.45453
 263.39536 258.14902 307.60123 255.91203 261.5959  296.41168 303.15024]
